In [1]:
# --- Before your go ----
# 1. Rename Assignment-03-###.ipynb where ### is your student ID.
# 2. The deadline of Assignment-03 is 23:59pm, 05-31-2023


# --- Explore HMM POS Taggers using Brown corpus ---
# In this assignment, you will explore two taggers for a Brown corpus.
# import your packages here

from nltk import FreqDist, MLEProbDist, LidstoneProbDist, ConditionalFreqDist, ConditionalProbDist
from nltk.util import unique_list
from nltk.tag import HiddenMarkovModelTagger
from collections import defaultdict, Counter

In [2]:
# Task 1 --- Load and explore your data ---
# 1). load train/test samples from Brown corpus files, brown-train.txt, brown-test.txt.
# 2). load all 12 tags from brown-tag.txt and print it out
# 3). counting how many sentences and words in both train and test datasets.
# 4). for each tag, counting how many words in train and test. e.g, tag1: [count_tr, count_te]

# Your code

In [3]:
def load_dataset(path, print_data=True):
    with open(path) as f:
        tmp=[item.strip().split("\t") for item in f.readlines()]
    dataset=[]
    for item in tmp:
        if(len(item)==1 and item[0]!=""):
            dataset.append([])
        elif(len(item)==2):
            dataset[-1].append(tuple((item[0].lower(),item[1]))) # 让word小写
    if(print_data):
        print(path)
        for i in range(5):
            for j in range(5):
                print(dataset[i][j],end="")
            print("...")
        print("...\n")
    return dataset

trainset=load_dataset("brown-train.txt", print_data=True)
testset=load_dataset("brown-test.txt", print_data=True)

brown-train.txt
('mr.', 'NOUN')('podger', 'NOUN')('had', 'VERB')('thanked', 'VERB')('him', 'PRON')...
('but', 'CONJ')('there', 'PRT')('seemed', 'VERB')('to', 'PRT')('be', 'VERB')...
('such', 'PRT')('an', 'DET')('instrument', 'NOUN')('is', 'VERB')('expected', 'VERB')...
('my', 'DET')('future', 'ADJ')('plans', 'NOUN')('are', 'VERB')('to', 'PRT')...
('we', 'PRON')('ran', 'VERB')('east', 'NOUN')('for', 'ADP')('about', 'ADV')...
...

brown-test.txt
('``', '.')("i've", 'PRT')('got', 'VERB')('cancer', 'NOUN')(',', '.')...
('the', 'DET')('need', 'NOUN')('to', 'PRT')('protect', 'VERB')('the', 'DET')...
('one', 'NUM')('of', 'ADP')('his', 'DET')('innovations', 'NOUN')('was', 'VERB')...
('she', 'PRON')('rubbed', 'VERB')('her', 'PRON')('eyes', 'NOUN')('and', 'CONJ')...
('``', '.')('get', 'VERB')('into', 'ADP')('your', 'DET')('hovel', 'NOUN')...
...



In [4]:
with open("brown-tag.txt","r") as f:
    tags=[tag.strip() for tag in f.readlines()]
print(tags)

['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


In [5]:
def cnt_sen_and_word(dataset):
    words=[word for item in dataset for word,lab in item]
    print("number of unique words (include punctuation):",len(set(words)))
    print("number of words (include punctuation):",len(words))
    print("number of sentence:", len(dataset))
    
cnt_sen_and_word(trainset)
cnt_sen_and_word(testset)

number of unique words (include punctuation): 44993
number of words (include punctuation): 928327
number of sentence: 45800
number of unique words (include punctuation): 23114
number of words (include punctuation): 232865
number of sentence: 11540


In [6]:
cnt_tr=dict.fromkeys(tags,0)
cnt_te=dict.fromkeys(tags,0)
for sent in trainset:
    for item in sent:
        cnt_tr[item[1]]+=1
for sent in testset:
    for item in sent:
        cnt_te[item[1]]+=1
for tag in tags:
    print(tag,":",[cnt_tr[tag], cnt_te[tag]])       

. : [117723, 29842]
ADJ : [66985, 16736]
ADP : [115752, 29014]
ADV : [44765, 11474]
CONJ : [30455, 7696]
DET : [109418, 27601]
NOUN : [220451, 55107]
NUM : [11921, 2953]
PRON : [39657, 9677]
PRT : [23889, 5940]
VERB : [146199, 36551]
X : [1112, 274]


In [7]:
# Task 2 --- Build a baseline method, namely, the most frequent tagger ---
#     If you can recall, we introduced a strong baseline method (See Dan's book in 
# https://web.stanford.edu/~jurafsky/slp3/ed3book_jan72023.pdf Page 164.),
#     where we label each word by using the most frequent-used tag associated with it.
# 1). find the most frequent class label for each word in the training data.
#     For example, {tr_word_1:tag_1,tr_word_2:tag_2,...}
# 2). use your built method to predict tags for both train and test datasets.
#     You should print out two values: the accuracies of train and test samples.
#     You would expect that the accuracy on train will be > 0.9 (but never = 1.0) and higher than on test.

# Notice: since there are unkown words in test samples. 
#  Following ways could handle this (choose one or create your own): 
#  1). mark all words that appear only once in the data with a "UNK-x" tag
#  2). tag every out-of-vocabulary word with the majority tag among all training samples.
#  3). find more methods in https://github.com/Adamouization/POS-Tagging-and-Unknown-Words

# Your code

In [8]:
# the majority tag among all training samples
max(cnt_tr.items(),key=lambda x:x[1])

('NOUN', 220451)

In [9]:
# compute the most frequent labels associated with each word
def find_freq_labels(dataset):
    word_labels = defaultdict(Counter)
    for sent in dataset:
        for item in sent:
            word_labels[item[0]][item[1]] += 1
    most_freq_labels = {}  
    for word, counter in word_labels.items():
        most_freq_labels[word] = counter.most_common(1)[0][0]
    return most_freq_labels

# compute accuracy (every out-of-vocabulary word with the majority tag among all training samples)
def most_freq_acc(dataset, most_freq_labels):
    correct=0
    total=0
    for sent in dataset:
        for item in sent:
            total+=1
            if(most_freq_labels.get(item[0],'NOUN')==item[1]):
                correct+=1
    return correct/total*100

In [10]:
most_freq_labels=find_freq_labels(trainset)
print("Accuracy on training set:", most_freq_acc(trainset,most_freq_labels))
print("Accuracy on test set:", most_freq_acc(testset,most_freq_labels))

Accuracy on training set: 95.56330904950518
Accuracy on test set: 94.53932536018723


In [11]:
# mark all words that appear only once in the data with "UNK"
def process_train_dataset(dataset):
    words=[]
    for sent in dataset:
        for word,tag in sent:
            words.append(word)
    vocab_freq=FreqDist(words)
    vocab=set(vocab_freq.keys())-set(vocab_freq.hapaxes())
    hapaxes=set(vocab_freq.hapaxes())
    new_dataset=[]
    for sent in dataset:
        out=[("<UNK>",tag) if word not in vocab else (word,tag) for word,tag in sent]
        new_dataset.append(out)
    return vocab, hapaxes, new_dataset

def process_test_dataset(dataset, hapaxes, vocab):
    new_dataset=[]
    for sent in dataset:
        out=[("<UNK>",tag) if word not in vocab else (word,tag) for word,tag in sent]
        new_dataset.append(out)
    return new_dataset

vocab, hapaxes, new_trainset=process_train_dataset(trainset)
new_testset=process_test_dataset(testset, hapaxes, vocab)

In [12]:
new_most_freq_labels=find_freq_labels(new_trainset)
print("Accuracy on training set:", most_freq_acc(new_trainset, new_most_freq_labels))
print("Accuracy on test set:", most_freq_acc(new_testset, new_most_freq_labels))

Accuracy on training set: 94.66416467473208
Accuracy on test set: 94.0141283576321


In [13]:
# Task 3 --- Build an HMM tagger ---
# 1) You should use nltk.tag.HiddenMarkovModelTagger to build an HMM tagger.
#    It has parameters: symbols, states, transitions, outputs, priors, transform (ignore it).
#    Specify these parameters properly. For example, you can use MLE to estimate transitions, outputs and priors.
#    That is, MLE to estimate matrix A (transition matrix), and matrix B (output probabilites) (See. Page 8.4.3)
# 2) After build your model, report both the accuracy of HMM tagger for train samples and test samples.
# 
# 3) Compared with your baseline method, discuss that why your HMM tagger is better/worse than baseline method.

# Notice: You may also need to handle unknown words just like Task 2.

# Your code

In [14]:
def gen_hmm_para_MLE(dataset):
    # symbols: seq of any
    symbols = unique_list(word for sent in dataset for word, tag in sent)

    # states: seq of any
    states = unique_list(tag for sent in dataset for word, tag in sent)  

    # transitions: ConditionalProbDistI
    _transitions=[]
    for sent in dataset:
        for i in range(len(sent)-1):
            _transitions.append((sent[i][1],sent[i+1][1]))
    transitions=ConditionalProbDist(ConditionalFreqDist(_transitions), MLEProbDist)
    
    # outputs: ConditionalProbDistI
    _outputs=[]
    for sent in dataset:
        for word,tag in sent:
            _outputs.append((tag, word))
    outputs=ConditionalProbDist(ConditionalFreqDist(_outputs), MLEProbDist)

    # prior: ProbDistI
    _priors=[]
    for sent in dataset:
        word,tag=sent[0]
        _priors.append(tag)
    priors=MLEProbDist(FreqDist(_priors))  

    return symbols, states, transitions, outputs, priors


In [15]:
symbols, states, transitions, outputs, priors=gen_hmm_para_MLE(trainset)
tagger1=HiddenMarkovModelTagger(symbols, states, transitions, outputs, priors)
tagger1.test(trainset)
tagger1.test(testset)

accuracy over 928327 tokens: 97.42
accuracy over 232865 tokens: 81.41


In [16]:
symbols, states, transitions, outputs, priors=gen_hmm_para_MLE(new_trainset)
tagger1=HiddenMarkovModelTagger(symbols, states, transitions, outputs, priors)
tagger1.test(new_trainset)
tagger1.test(new_testset)

accuracy over 928327 tokens: 96.57
accuracy over 232865 tokens: 95.86


| Tagger | Accuracy on trainset | Accuracy on testset |
| --- | --- | --- |
| Baseline+MostFreq | 95.56 | 94.53 |
| Baseline+MostFreq+UNK | <font color=green>94.66</font> | 94.01 |
| HMM+MLE | <font color=red>97.42</font> | <font color=green>81.41</font>|
| HMM+MLE+UNK | 96.57 | <font color=red>95.86</font> |

Baseline标注器使用UNK处理低频词后在训练集和测试集上表现均有所下降，这是因为使用unk处理后丢失了原先数据集中的信息。

HMM标注器的MLE方法在我们的实验中产生过拟合问题，在训练集表现最好，在测试集表现最差，这是因为它会给未出现的事件分配零概率，这样就会导致一些标记永远不会被预测到。

UNK处理后HMM缓解了过拟合，能够改善模型对未知词汇和低频词的识别能力，并通过观察其上下文信息来预测其标注，从而提高模型总体性能，在测试集表现最好。

下面对MLE方法进行平滑，看是否会改善HMM的表现。

In [17]:
def gen_hmm_para_Lidstone(dataset):
    estimator = lambda fdist, bins: LidstoneProbDist(fdist, 0.1, bins)
    # estimator = lambda fdist, bins: MLEProbDist(fdist)
    
    # symbols: seq of any
    symbols = unique_list(word for sent in dataset for word, tag in sent)

    # states: seq of any
    states = unique_list(tag for sent in dataset for word, tag in sent)  

    # transitions: ConditionalProbDistI
    _transitions=[]
    for sent in dataset:
        for i in range(len(sent)-1):
            _transitions.append((sent[i][1],sent[i+1][1]))
    transitions=ConditionalProbDist(ConditionalFreqDist(_transitions), estimator, len(states))
    
    # outputs: ConditionalProbDistI
    _outputs=[]
    for sent in dataset:
        for word,tag in sent:
            _outputs.append((tag, word))
    outputs=ConditionalProbDist(ConditionalFreqDist(_outputs), estimator, len(symbols))

    # prior: ProbDistI
    _priors=[]
    for sent in dataset:
        word,tag=sent[0]
        _priors.append(tag)
    priors=estimator(FreqDist(_priors), len(states))  

    return symbols, states, transitions, outputs, priors


In [18]:
symbols, states, transitions, outputs, priors=gen_hmm_para_Lidstone(trainset)
tagger1=HiddenMarkovModelTagger(symbols, states, transitions, outputs, priors)
tagger1.test(trainset)
tagger1.test(testset)

accuracy over 928327 tokens: 96.95
accuracy over 232865 tokens: 95.33


In [19]:
symbols, states, transitions, outputs, priors=gen_hmm_para_Lidstone(new_trainset)
tagger1=HiddenMarkovModelTagger(symbols, states, transitions, outputs, priors)
tagger1.test(new_trainset)
tagger1.test(new_testset)

accuracy over 928327 tokens: 96.35
accuracy over 232865 tokens: 95.72


| Tagger | Accuracy on trainset | Accuracy on testset |
| --- | --- | --- |
| Baseline+MostFreq | 95.56 | 94.53 |
| Baseline+MostFreq+UNK | <font color=green>94.66</font> | 94.01 |
| HMM+MLE | <font color=red>97.42</font> | <font color=green>81.41</font>|
| HMM+MLE+UNK | 96.57 | <font color=red>95.86</font> |
| HMM+Lidstone | 96.95 | 95.33|
| HMM+Lidstone+UNK | 96.35 | 95.72 |

可以看到Lidstone smoothing方法提升了不使用UNK处理的测试集上的表现，这是因为通过将每个计数增加一个平滑常量来避免了过拟合。而同时使用Lidstone Smoothing和UNK可能会导致过度平滑，从而降低标注器的预测准确性。